In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

In [27]:
genreList = []
removedCharacters = ["'", '[', ']', ' ']
cutOffScore = 5

In [3]:
def genreEnumerator(lis):
    for x in range(len(lis)):
        if lis[x] in genreList:
            lis[x] = genreList.index(lis[x])
        else:
            genreList.append(lis[x])
            lis[x] = genreList.index(lis[x])
    while len(lis) < 8:
        lis.append(-1)
    return lis

In [4]:
def stringToList(string):
    new = string
    for chara in removedCharacters:
        new = new.replace(chara, '')
    return new.split(',')

In [5]:
def extractmonth(string):
    lis = string.split('-')
    return float(lis[1])

In [6]:
dataset = pd.read_csv('movieDataset.csv')

In [28]:
train, test = train_test_split(dataset, test_size=0.2)

In [29]:
datasetRef = train.filter(['Title', 'Vote Count', 'Release Date'], axis=1)
dataGenre = train.filter(['Genre'], axis=1)
dataGenre = dataGenre['Genre'].apply(stringToList)
dataGenre.apply(genreEnumerator)
datGenre = pd.DataFrame(columns=['Genre 1', 'Genre 2', 'Genre 3', 'Genre 4', 'Genre 5', 'Genre 6', 'Genre 7', 'Genre 8'], data=dataGenre.tolist(), index=dataGenre.index)
datasetRating = train.filter(['Vote Average'], axis=1)
overviewRef = train['Overview']
overviewRef = overviewRef.str.count(' ') + 1
overviewRef = pd.DataFrame(overviewRef)
datasetRating = datasetRating.squeeze()

In [30]:
datasetRef['OverviewWordCount'] = overviewRef
datasetRef['Title'] = datasetRef['Title'].apply(len)

In [31]:
datasetRef= datasetRef.join(datGenre)
datasetRef['Release Date'] = datasetRef['Release Date'].apply(extractmonth)

In [32]:
datasetRating = datasetRating.apply(lambda x: x > cutOffScore)

In [33]:
mlp = MLPClassifier(max_iter = 200, solver='adam', tol=1e-5, hidden_layer_sizes=(20,))

In [34]:
mlp.fit(datasetRef, datasetRating)

MLPClassifier(hidden_layer_sizes=(20,), tol=1e-05)

In [35]:
testsetRef = test.filter(['Title', 'Vote Count', 'Release Date'], axis=1)

testGenre = test.filter(['Genre'], axis=1)
testGenre = testGenre['Genre'].apply(stringToList)
testGenre.apply(genreEnumerator)
tesGenre = pd.DataFrame(columns=['Genre 1', 'Genre 2', 'Genre 3', 'Genre 4', 'Genre 5', 'Genre 6', 'Genre 7', 'Genre 8'], data=testGenre.tolist(), index=testGenre.index)
testsetRating = test.filter(['Vote Average'], axis=1)
testviewRef = test['Overview']
testviewRef = testviewRef.str.count(' ') + 1
testviewRef = pd.DataFrame(testviewRef)
testsetRating = testsetRating.squeeze()
testsetRef['OverviewWordCount'] = testviewRef
testsetRef['Title'] = testsetRef['Title'].apply(len)

testsetRef= testsetRef.join(tesGenre)

testsetRef['Release Date'] = testsetRef['Release Date'].apply(extractmonth)
testsetRating = testsetRating.apply(lambda x: x > cutOffScore)

In [36]:
mlp.score(testsetRef, testsetRating)

0.8025622481661329

In [37]:
testsetRating.value_counts(normalize=True)

True     0.6106
False    0.3894
Name: Vote Average, dtype: float64